## Table Linking Pipeline

This notebook showcases the pipeline for linking entities in a table to wikidata. For the purpose of this tutorial, I use a small table.<br> 
The steps involved in this notebook are as follows:<br>
    a) Generate candidates using fuzzy augmented method and exact matches method present in table linker<br>
    b) Rank the candidates using centroid-singleton strategy using table linker commands. 

In [15]:
import pandas as pd
import os
import glob
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

import numpy as np
import sklearn.metrics

pd.reset_option('all')

As the xlwt package is no longer maintained, the xlwt engine will be removed in a future version of pandas. This is the only engine in pandas that supports writing in the xls format. Install openpyxl and write to an xlsx file instead.

: boolean
    use_inf_as_null had been deprecated and will be removed in a future
    version. Use `use_inf_as_na` instead.



In [3]:
table_path = '../../'
table_name = 'players.csv'

In [4]:
p = os.path.join(table_path,table_name)
pd.read_csv(p,nrows = 10)

,Rank,Player (2011 TBs),TBs
0,1,Hank Aaron,"6,856"
1,2,Stan Musial,"6,134"
2,3,Willie Mays,"6,066"
3,4,Barry Bonds,"5,976"
4,5,Ty Cobb,"5,854"
5,6,Babe Ruth,"5,793"
6,7,Pete Rose,"5,752"
7,8,Carl Yastrzemski,"5,539"
8,9,Eddie Murray,"5,397"
9,10,Rafael Palmeiro,"5,388"


We will use this table and link the column "Player (2011 TBs)" to wikidata. <br>

Before start the process of candidate generation, we need to convert the table into a canonical format

### Canonicalize the input table

In [5]:
canonical_file = os.path.join(table_path,'canonical-players.csv')
!tl canonicalize -c "Player (2011 TBs)" $p > $canonical_file

In [6]:
pd.read_csv(canonical_file,nrows = 10)

,column,row,label
0,1,0,Hank Aaron
1,1,1,Stan Musial
2,1,2,Willie Mays
3,1,3,Barry Bonds
4,1,4,Ty Cobb
5,1,5,Babe Ruth
6,1,6,Pete Rose
7,1,7,Carl Yastrzemski
8,1,8,Eddie Murray
9,1,9,Rafael Palmeiro


### Candidate Generation

In [8]:
cand_file = os.path.join(table_path,'cand-players.csv')
gt_file = os.path.join(table_path,'players-gt.csv')

In [11]:
%%time
!tl clean -c label -o label_clean $canonical_file / get-fuzzy-augmented-matches -c label_clean > $cand_file

CPU times: user 169 ms, sys: 67.2 ms, total: 236 ms
Wall time: 11.5 s


In [9]:
gt_file = os.path.join(table_path,'players-gt.csv')
gt_labeled_file = os.path.join(table_path,'players-gt-labeled.csv')

In [10]:
!tl --url http://kg2018a.isi.edu:9200 --index wiki_labels_aliases_3 get-exact-matches -c label_clean $cand_file \
        / ground-truth-labeler --gt-file $gt_file > $gt_labeled_file

In [11]:
pd.read_csv(gt_labeled_file,nrows = 10)

,column,row,label,label_clean,kg_id,kg_labels,method,retrieval_score,GT_kg_id,GT_kg_label,evaluation_label
0,1,0,Hank Aaron,Hank Aaron,Q215777,Hank Aaron,fuzzy-augmented,37.630530,Q215777,Hank Aaron,1
1,1,0,Hank Aaron,Hank Aaron,Q47513596,Hank Aaron,fuzzy-augmented,16.903837,Q215777,Hank Aaron,-1
2,1,0,Hank Aaron,Hank Aaron,Q1518478,Hank Aaron Award,fuzzy-augmented,19.805542,Q215777,Hank Aaron,-1
3,1,0,Hank Aaron,Hank Aaron,Q14679126,Hank Aaron Stadium,fuzzy-augmented,28.061468,Q215777,Hank Aaron,-1
4,1,0,Hank Aaron,Hank Aaron,Q28453830,Hank Aaron State Trail,fuzzy-augmented,26.173532,Q215777,Hank Aaron,-1
5,1,0,Hank Aaron,Hank Aaron,Q92433937,Reflections on Hank Aaron,fuzzy-augmented,26.173532,Q215777,Hank Aaron,-1
6,1,0,Hank Aaron,Hank Aaron,Q6665277,Template:AL Hank Aaron Award Winners,fuzzy-augmented,24.523617,Q215777,Hank Aaron,-1
7,1,0,Hank Aaron,Hank Aaron,Q5648263,Hank Aaron: Chasing the Dream,fuzzy-augmented,24.523617,Q215777,Hank Aaron,-1
8,1,0,Hank Aaron,Hank Aaron,Q8853836,Template:NL Hank Aaron Award Winners,fuzzy-augmented,24.523617,Q215777,Hank Aaron,-1
9,1,0,Hank Aaron,Hank Aaron,Q66847614,President Carter with Hank Aaron (NAID 180805),fuzzy-augmented,21.777962,Q215777,Hank Aaron,-1


### Candidate Ranking

We calculate scores using text-embedding and graph-embedding for evaluating which of the embeddings produces better ranking. 

In [12]:
output_file = os.path.join(table_path, 'players-embedding-scores.csv')

In [13]:
!tl score-using-embedding -o text-embedding-score --embedding-url http://kg2018a.isi.edu:9200/wikidataos-text-embedding-01/doc/ $gt_labeled_file\
    / score-using-embedding -o graph-embedding-score --embedding-url http://kg2018a.isi.edu:9200/wikidataos-graph-embedding-01/doc/ \
                            > $output_file

In [18]:
df = pd.read_csv(output_file)
df.head()

,column,row,label,label_clean,kg_id,kg_labels,method,retrieval_score,GT_kg_id,GT_kg_label,evaluation_label,text-embedding-score,graph-embedding-score
0,1,0,Hank Aaron,Hank Aaron,Q215777,Hank Aaron,fuzzy-augmented,37.630530,Q215777,Hank Aaron,1,0.961772,0.839375
1,1,0,Hank Aaron,Hank Aaron,Q47513596,Hank Aaron,fuzzy-augmented,16.903837,Q215777,Hank Aaron,-1,0.501684,0.356457
2,1,0,Hank Aaron,Hank Aaron,Q1518478,Hank Aaron Award,fuzzy-augmented,19.805542,Q215777,Hank Aaron,-1,0.710425,0.552475
3,1,0,Hank Aaron,Hank Aaron,Q14679126,Hank Aaron Stadium,fuzzy-augmented,28.061468,Q215777,Hank Aaron,-1,0.596523,0.432508
4,1,0,Hank Aaron,Hank Aaron,Q28453830,Hank Aaron State Trail,fuzzy-augmented,26.173532,Q215777,Hank Aaron,-1,0.507785,NaN


### Evaluation

In [16]:
# parse eval file
from pandas.core.common import SettingWithCopyError

pd.options.mode.chained_assignment = 'raise'

def parse_eval_files_stats(eval_data):
    res = {}
    candidate_eval_data = eval_data.groupby(['row', 'column'])['row'].count().reset_index(name="count")
    res['num_tasks'] = len(eval_data.groupby(['row', 'column']))
    res['num_tasks_with_gt'] = len(eval_data[pd.notna(eval_data['GT_kg_id'])].groupby(['row', 'column']))
    res['num_tasks_with_gt_in_candidate'] = len(eval_data[eval_data['evaluation_label'] == 1].groupby(['row', 'column']))
    res['num_tasks_with_singleton_candidate'] = len(candidate_eval_data[candidate_eval_data['count'] == 1].groupby(['row', 'column']))
    
    singleton_eval_data = candidate_eval_data[candidate_eval_data['count'] == 1]
    num_tasks_with_singleton_candidate_with_gt = 0
    for i, row in singleton_eval_data.iterrows():
        row_idx, col_idx = row['row'], row['column']
        c_e_data = eval_data[(eval_data['row'] == row_idx) & (eval_data['column'] == col_idx)]
        assert len(c_e_data) == 1
        if c_e_data.iloc[0]['evaluation_label'] == 1:
            num_tasks_with_singleton_candidate_with_gt += 1
    res['num_tasks_with_singleton_candidate_with_gt'] = num_tasks_with_singleton_candidate_with_gt
    
    num_tasks_with_retrieval_top_one_accurate = []
    num_tasks_with_retrieval_top_five_accurate = []
    num_tasks_with_retrieval_top_ten_accurate = []
    num_tasks_with_text_top_one_accurate = []
    num_tasks_with_text_top_five_accurate = []
    num_tasks_with_text_top_ten_accurate = []
    num_tasks_with_graph_top_one_accurate = []
    num_tasks_with_graph_top_five_accurate = []
    num_tasks_with_graph_top_ten_accurate = []
    ndcg_score_r_list = []
    ndcg_score_t_list = []
    ndcg_score_g_list = []
    has_gt_list = []
    has_gt_in_candidate = []
    # candidate_eval_data = candidate_eval_data[:1]
    for i, row in candidate_eval_data.iterrows():
        #print(i)
        row_idx, col_idx = row['row'], row['column']
        c_e_data = eval_data[(eval_data['row'] == row_idx) & (eval_data['column'] == col_idx)]
        assert len(c_e_data) > 0
        
        if np.nan not in set(c_e_data['GT_kg_id']):
            has_gt_list.append(1)
        else:
            has_gt_list.append(0)
            
        if 1 in set(c_e_data['evaluation_label']):
            has_gt_in_candidate.append(1)
        else:
            has_gt_in_candidate.append(0)
        
        # handle retrieval score
        s_data = c_e_data.sort_values(by=['retrieval_score'], ascending=False)
        if s_data.iloc[0]['evaluation_label'] == 1:
            num_tasks_with_retrieval_top_one_accurate.append(1)
        else:
            num_tasks_with_retrieval_top_one_accurate.append(0)
        if 1 in set(s_data.iloc[0:5]['evaluation_label']):
            num_tasks_with_retrieval_top_five_accurate.append(1)
        else:
            num_tasks_with_retrieval_top_five_accurate.append(0)
        if 1 in set(s_data.iloc[0:10]['evaluation_label']):
            num_tasks_with_retrieval_top_ten_accurate.append(1)
        else:
            num_tasks_with_retrieval_top_ten_accurate.append(0)
        
        
        # handle text-embedding-score
        s_data = c_e_data.sort_values(by=['text-embedding-score'], ascending=False)
        if s_data.iloc[0]['evaluation_label'] == 1:
            num_tasks_with_text_top_one_accurate.append(1)
        else:
            num_tasks_with_text_top_one_accurate.append(0)
        if 1 in set(s_data.iloc[0:5]['evaluation_label']):
            num_tasks_with_text_top_five_accurate.append(1)
        else:
            num_tasks_with_text_top_five_accurate.append(0)
        if 1 in set(s_data.iloc[0:10]['evaluation_label']):
            num_tasks_with_text_top_ten_accurate.append(1)
        else:
            num_tasks_with_text_top_ten_accurate.append(0)
            
        # handle graph-embedding-score
        s_data = c_e_data.sort_values(by=['graph-embedding-score'], ascending=False)
        if s_data.iloc[0]['evaluation_label'] == 1:
            num_tasks_with_graph_top_one_accurate.append(1)
        else:
            num_tasks_with_graph_top_one_accurate.append(0)
        if 1 in set(s_data.iloc[0:5]['evaluation_label']):
            num_tasks_with_graph_top_five_accurate.append(1)
        else:
            num_tasks_with_graph_top_five_accurate.append(0)
        if 1 in set(s_data.iloc[0:10]['evaluation_label']):
            num_tasks_with_graph_top_ten_accurate.append(1)
        else:
            num_tasks_with_graph_top_ten_accurate.append(0)
        
        cf_e_data = c_e_data.copy()
        cf_e_data['evaluation_label'] = cf_e_data['evaluation_label'].replace(-1, 0)
        cf_e_data['text-embedding-score'] = cf_e_data['text-embedding-score'].replace(np.nan, 0)
        cf_e_data['graph-embedding-score'] = cf_e_data['graph-embedding-score'].replace(np.nan, 0)

        try:
            ndcg_score_t_list.append(
                sklearn.metrics.ndcg_score(
                    np.array([list(cf_e_data['evaluation_label'])]),
                    np.array([list(cf_e_data['text-embedding-score'])])
                )
            )
        except:
            if len(cf_e_data['evaluation_label']) == 1 and cf_e_data['evaluation_label'].iloc[0] == 1:
                ndcg_score_t_list.append(1.0)
            elif len(cf_e_data['evaluation_label']) == 1 and cf_e_data['evaluation_label'].iloc[0] != 1:
                ndcg_score_t_list.append(0.0)
            else:
                print("text", cf_e_data['evaluation_label'], cf_e_data['text-embedding-score'] )
                print("why am i here")
        try:
            ndcg_score_g_list.append(
                sklearn.metrics.ndcg_score(
                    np.array([list(cf_e_data['evaluation_label'])]),
                    np.array([list(cf_e_data['graph-embedding-score'])])
                )
            )
        except:
            if len(cf_e_data['evaluation_label']) == 1 and cf_e_data['evaluation_label'].iloc[0] == 1:
                ndcg_score_g_list.append(1.0)
            elif len(cf_e_data['evaluation_label']) == 1 and cf_e_data['evaluation_label'].iloc[0] != 1:
                ndcg_score_g_list.append(0.0)
            else:
                print("graph", cf_e_data['evaluation_label'], cf_e_data['graph-embedding-score'])
                print("why am i here")

    #candidate_eval_data['r_ndcg'] = ndcg_score_r_list
    candidate_eval_data['t_ndcg'] = ndcg_score_t_list
    candidate_eval_data['g_ndcg'] = ndcg_score_g_list
    candidate_eval_data['retrieval_top_one_accurate'] = num_tasks_with_retrieval_top_one_accurate
    candidate_eval_data['retrieval_top_five_accurate'] = num_tasks_with_retrieval_top_five_accurate
    candidate_eval_data['retrieval_top_ten_accurate'] = num_tasks_with_retrieval_top_ten_accurate
    candidate_eval_data['text_top_one_accurate'] = num_tasks_with_text_top_one_accurate
    candidate_eval_data['text_top_five_accurate'] = num_tasks_with_text_top_five_accurate
    candidate_eval_data['text_top_ten_accurate'] = num_tasks_with_text_top_ten_accurate
    candidate_eval_data['graph_top_one_accurate'] = num_tasks_with_graph_top_one_accurate
    candidate_eval_data['graph_top_five_accurate'] = num_tasks_with_graph_top_five_accurate
    candidate_eval_data['graph_top_ten_accurate'] = num_tasks_with_graph_top_five_accurate
    candidate_eval_data['has_gt'] = has_gt_list
    candidate_eval_data['has_gt_in_candidate'] = has_gt_in_candidate
            
    res['num_tasks_with_retrieval_top_one_accurate'] = sum(num_tasks_with_retrieval_top_one_accurate)
    res['num_tasks_with_retrieval_top_five_accurate'] = sum(num_tasks_with_retrieval_top_five_accurate)
    res['num_tasks_with_retrieval_top_ten_accurate'] = sum(num_tasks_with_retrieval_top_ten_accurate)
    res['num_tasks_with_text_top_one_accurate'] = sum(num_tasks_with_text_top_one_accurate)
    res['num_tasks_with_text_top_five_accurate'] = sum(num_tasks_with_text_top_five_accurate)
    res['num_tasks_with_text_top_ten_accurate'] = sum(num_tasks_with_text_top_ten_accurate)
    res['num_tasks_with_graph_top_one_accurate'] = sum(num_tasks_with_graph_top_one_accurate)
    res['num_tasks_with_graph_top_five_accurate'] = sum(num_tasks_with_graph_top_five_accurate)
    res['num_tasks_with_graph_top_ten_accurate'] = sum(num_tasks_with_graph_top_ten_accurate)
            
    return res, candidate_eval_data

In [20]:
res, candidate_eval_data = parse_eval_files_stats(df)

In [21]:
print(res)
display(candidate_eval_data)

{'num_tasks': 27, 'num_tasks_with_gt': 27, 'num_tasks_with_gt_in_candidate': 27, 'num_tasks_with_singleton_candidate': 0, 'num_tasks_with_singleton_candidate_with_gt': 0, 'num_tasks_with_retrieval_top_one_accurate': 13, 'num_tasks_with_retrieval_top_five_accurate': 17, 'num_tasks_with_retrieval_top_ten_accurate': 22, 'num_tasks_with_text_top_one_accurate': 18, 'num_tasks_with_text_top_five_accurate': 26, 'num_tasks_with_text_top_ten_accurate': 27, 'num_tasks_with_graph_top_one_accurate': 17, 'num_tasks_with_graph_top_five_accurate': 27, 'num_tasks_with_graph_top_ten_accurate': 27}


,row,column,count,t_ndcg,g_ndcg,retrieval_top_one_accurate,retrieval_top_five_accurate,retrieval_top_ten_accurate,text_top_one_accurate,text_top_five_accurate,text_top_ten_accurate,graph_top_one_accurate,graph_top_five_accurate,graph_top_ten_accurate,has_gt,has_gt_in_candidate
0,0,1,111,1.000000,0.693426,1,1,1,1,1,1,0,1,1,1,1
1,1,1,103,1.000000,1.000000,0,0,0,1,1,1,1,1,1,1,1
2,2,1,107,1.000000,1.000000,0,0,1,1,1,1,1,1,1,1,1
3,3,1,122,1.000000,0.693426,1,1,1,1,1,1,0,1,1,1,1
4,4,1,130,1.000000,1.000000,0,0,1,1,1,1,1,1,1,1,1
5,5,1,108,0.693426,0.501266,1,1,1,0,1,1,0,1,1,1,1
6,6,1,122,1.000000,1.000000,0,0,1,1,1,1,1,1,1,1,1
7,7,1,101,1.000000,1.000000,1,1,1,1,1,1,1,1,1,1,1
8,8,1,119,1.000000,1.000000,1,1,1,1,1,1,1,1,1,1,1
9,9,1,103,0.693426,1.000000,0,0,0,0,1,1,1,1,1,1,1


In [22]:
# Conclusion of exact-match on all tasks with ground truth (no filtering)
print(f"number of tasks: {res['num_tasks']}")
print(f"number of tasks with ground truth: {res['num_tasks_with_gt']}")
print(f"number of tasks with ground truth in candidate set: {res['num_tasks_with_gt_in_candidate']}, which is {res['num_tasks_with_gt_in_candidate']/res['num_tasks_with_gt'] * 100}%")
print(f"number of tasks has singleton candidate set: {res['num_tasks_with_singleton_candidate']}, which is {res['num_tasks_with_singleton_candidate']/res['num_tasks_with_gt'] * 100}%")
print(f"number of tasks has singleton candidate set which is ground truth: {res['num_tasks_with_singleton_candidate_with_gt']}, which is {res['num_tasks_with_singleton_candidate_with_gt']/res['num_tasks_with_gt'] * 100}%")
print()
#print(f"number of tasks with top-1 accuracy in terms of retrieval score: {res['num_tasks_with_retrieval_top_one_accurate']}, which is {res['num_tasks_with_retrieval_top_one_accurate']/res['num_tasks_with_gt'] * 100}%")
#print(f"number of tasks with top-5 accuracy in terms of retrieval score: {res['num_tasks_with_retrieval_top_five_accurate']}, which is {res['num_tasks_with_retrieval_top_five_accurate']/res['num_tasks_with_gt'] * 100}%")
#print(f"number of tasks with top-10 accuracy in terms of retrieval score: {res['num_tasks_with_retrieval_top_ten_accurate']}, which is {res['num_tasks_with_retrieval_top_ten_accurate']/res['num_tasks_with_gt'] * 100}%")
#print()
print(f"number of tasks with top-1 accuracy in terms of text embedding score: {res['num_tasks_with_text_top_one_accurate']}, which is {res['num_tasks_with_text_top_one_accurate']/res['num_tasks_with_gt'] * 100}%")
print(f"number of tasks with top-5 accuracy in terms of text embedding score: {res['num_tasks_with_text_top_five_accurate']}, which is {res['num_tasks_with_text_top_five_accurate']/res['num_tasks_with_gt'] * 100}%")
print(f"number of tasks with top-10 accuracy in terms of text embedding score: {res['num_tasks_with_text_top_ten_accurate']}, which is {res['num_tasks_with_text_top_ten_accurate']/res['num_tasks_with_gt'] * 100}%")
print()
print(f"number of tasks with top-1 accuracy in terms of graph embedding score: {res['num_tasks_with_graph_top_one_accurate']}, which is {res['num_tasks_with_graph_top_one_accurate']/res['num_tasks_with_gt'] * 100}%")
print(f"number of tasks with top-5 accuracy in terms of graph embedding score: {res['num_tasks_with_graph_top_five_accurate']}, which is {res['num_tasks_with_graph_top_five_accurate']/res['num_tasks_with_gt'] * 100}%")
print(f"number of tasks with top-10 accuracy in terms of graph embedding score: {res['num_tasks_with_graph_top_ten_accurate']}, which is {res['num_tasks_with_graph_top_ten_accurate']/res['num_tasks_with_gt'] * 100}%")
print()
candidate_eval_data_with_gt = candidate_eval_data[candidate_eval_data['has_gt'] == 1]
#print(f"average ndcg score ranked by retrieval score: {candidate_eval_data_with_gt['r_ndcg'].mean()}")
print(f"average ndcg score ranked by text-embedding-score: {candidate_eval_data_with_gt['t_ndcg'].mean()}")
print(f"average ndcg score ranked by graph-embedding-score: {candidate_eval_data_with_gt['g_ndcg'].mean()}")

number of tasks: 27
number of tasks with ground truth: 27
number of tasks with ground truth in candidate set: 27, which is 100.0%
number of tasks has singleton candidate set: 0, which is 0.0%
number of tasks has singleton candidate set which is ground truth: 0, which is 0.0%

number of tasks with top-1 accuracy in terms of text embedding score: 18, which is 66.66666666666666%
number of tasks with top-5 accuracy in terms of text embedding score: 26, which is 96.29629629629629%
number of tasks with top-10 accuracy in terms of text embedding score: 27, which is 100.0%

number of tasks with top-1 accuracy in terms of graph embedding score: 17, which is 62.96296296296296%
number of tasks with top-5 accuracy in terms of graph embedding score: 27, which is 100.0%
number of tasks with top-10 accuracy in terms of graph embedding score: 27, which is 100.0%

average ndcg score ranked by text-embedding-score: 0.8684765115997537
average ndcg score ranked by graph-embedding-score: 0.8591191183147332